In [ ]:
"""
Make plots of each unit, but not uisng raseter, just FR, which makes it much easier, as you 
can use DFallpa, which holds sm fr, but not raster.
|
11/11/25 - made for MS2 (syntax).

"""

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load dataset

In [ ]:
SAVEDIR = f"/lemur2/lucas/analyses/recordings/main/syntax_good"


In [ ]:
from neuralmonkey.classes.population_mult import load_handsaved_wrapper, dfpa_concat_merge_pa_along_trials, dfpa_concatbregion_preprocess_wrapper, dfpa_concat_bregion_to_combined_bregion
from pythonlib.tools.exceptions import NotEnoughDataException
SAVEDIR = f"/lemur2/lucas/analyses/recordings/main/syntax_good"

# Main
animal = "Diego"
date = 230913

# Two shapes
# animal = "Diego"
# date = 240822

version = "stroke"
combine = False

from neuralmonkey.classes.population_mult import load_handsaved_wrapper, dfpa_concat_bregion_to_combined_bregion

### (1) load Grammar Dfallpa
question = "RULE_ANBMCK_STROKE"
DFallpa = load_handsaved_wrapper(animal, date, version=version, combine_areas=combine, 
                                    question=question)
DFallpa = dfpa_concat_bregion_to_combined_bregion(DFallpa)

if False:
    try:
        ### (2) Load SP data
        _question = "SP_BASE_stroke"
        _twind = [-0.5, 2.1]
        DFallpaSP = load_handsaved_wrapper(animal, date, version=version, combine_areas=combine, 
                                            question=_question, twind=_twind)
        DFallpaSP = dfpa_concat_bregion_to_combined_bregion(DFallpaSP)

        # Merge SP and grammar along chan indices
        DFallpa = dfpa_concat_merge_pa_along_trials(DFallpa, DFallpaSP)
        del DFallpaSP
    except NotEnoughDataException as err:
        # pass
        # Then this is because chans dont line up. Not good.
        raise err
    except Exception as err:
        pass

# Make a copy of all PA before normalization
fr_mean_subtract_method = "raw_fr"
dfpa_concatbregion_preprocess_wrapper(DFallpa, animal, date, fr_mean_subtract_method=fr_mean_subtract_method)


In [ ]:
from pythonlib.tools.plottools import savefig
SAVEDIR_PLOTS = f"{SAVEDIR}/smfr_each_unit/{animal}-{date}"
import os
os.makedirs(SAVEDIR_PLOTS, exist_ok=True)

# Plot smoothed fr, one figure for each neuron

In [ ]:
from neuralmonkey.scripts.analy_syntax_good_smfr_units import _extract_pa
pa = _extract_pa(DFallpa, bregion, True)

In [ ]:
pa.Xlabels["trials"]

### Grammar vs. SP

In [ ]:
from neuralmonkey.scripts.analy_syntax_good_smfr_units import plotwrap_grammar_vs_sp
plotwrap_grammar_vs_sp(DFallpa, SAVEDIR_PLOTS)

### Rank within chunk


In [ ]:
from neuralmonkey.scripts.analy_syntax_good_smfr_units import plotwrap_rank_within_chunk
plotwrap_rank_within_chunk(DFallpa, SAVEDIR_PLOTS)

### Two shapes

In [ ]:
DFallpa

In [ ]:
# Only run this if the need exists

In [ ]:
from neuralmonkey.scripts.analy_euclidian_dist_pop_script_MULT import load_preprocess_get_dates
list_dates, _, _, _ = load_preprocess_get_dates(animal, "two_shape_sets")
list_dates = list(set(list_dates))

if date in list_dates:
    from neuralmonkey.scripts.analy_syntax_good_smfr_units import plotwrap_two_shapes
    plotwrap_two_shapes(DFallpa, SAVEDIR_PLOTS)

### Heatmap (get a scalar fr)

In [ ]:
from neuralmonkey.scripts.analy_syntax_good_smfr_units import _extract_pa


In [ ]:
bregion = "preSMA"
pa = _extract_pa(DFallpa, bregion, True)

In [ ]:
twind_scal = (-0.1, 0.3)
pa = pa.slice_by_dim_values_wrapper("times", twind_scal)
pa_scal = pa.agg_wrapper("times")
pa_scal

In [ ]:
chan = 1346
ind_chan = pa_scal.Chans.index(chan)
frvec = pa_scal.X[ind_chan, :, 0]


In [ ]:
from pythonlib.tools.pandastools import plot_subplots_heatmap
dflab = pa_scal.Xlabels["trials"]
# add in the fr
dflab["fr_scal"] = frvec

In [ ]:
# plot_subplots_heatmap(dflab, "crg_shape", "chunk_within_rank_fromlast", "fr_scal", "task_kind", False, True)
plot_subplots_heatmap(dflab, "crg_shape", "chunk_within_rank_semantic_v3", "fr_scal", "task_kind", False, True)


# Plot all trials (ie heatmap, with row = trials)

In [ ]:
from neuralmonkey.scripts.analy_syntax_good_smfr_units import _extract_pa

bregion = "preSMA"
pa = _extract_pa(DFallpa, bregion, True)
pa.Chans.index(1346)
var_effect = "chunk_within_rank_fromlast"
var_other = "crg_n"

SAVEDIR = "/tmp/FR_RASTER"
os.makedirs(SAVEDIR, exist_ok=True)
list_twind_scal_eucl = [(-0.1, 0.3)]
LIST_VAR_SS = ["chunk_within_rank_fromlast"]
LIST_VARS_OTHERS_SS = [["crg_n"]]
list_dims = [15]

pa.plot_heatmap_state_euclidean_wrapper(var_effect, var_other, SAVEDIR, 
                                             list_twind_scal_eucl, 
                                             LIST_VAR_SS, LIST_VARS_OTHERS_SS, list_dims=list_dims, ndims_timecourse=1,
                                             twind_base = (-0.6, 0.6),
                                             do_heatmap=True, do_state_space=True, do_euclidean=False,
                                             quick_mode=False)


In [ ]:
SAVEDIR_PLOTS

In [ ]:
# GOOD (I think)
from neuralmonkey.scripts.analy_syntax_good_smfr_units import plotwrap_rank_within_chunk_trialheatmaps
plotwrap_rank_within_chunk_trialheatmaps(DFallpa, SAVEDIR_PLOTS)


# Boundary neurons -- better controlled

### Sm fr, show effects of rank within, controlling for three motor parameters

In [ ]:
from neuralmonkey.scripts.analy_syntax_good_smfr_units import _preprocess_motor_params

In [ ]:
for pa in DFallpa["pa"]:
    ### Define new variables to bin the motor parameters to fewer bins
    _preprocess_motor_params(pa)


In [ ]:
from neuralmonkey.scripts.analy_syntax_good_smfr_units import plotwrap_rank_within_chunk_controlling_motor
SAVEDIR_PLOTS = "/tmp"
plotwrap_rank_within_chunk_controlling_motor(DFallpa, SAVEDIR_PLOTS)

### Quantifying single units

In [ ]:
from neuralmonkey.scripts.analy_syntax_good_smfr_units import plotwrap_scalar_rankwithin_controlling_motor
savedir = "/tmp/scal"
os.makedirs(savedir, exist_ok=True)
plotwrap_scalar_rankwithin_controlling_motor(DFallpa, savedir)

### Regression, predict activity for each unit

In [ ]:
# Get continuous motor variables
from neuralmonkey.scripts.analy_syntax_good_eucl_state import preprocess_dfallpa_motor_features    
preprocess_dfallpa_motor_features(DFallpa, do_zscore=True)


In [ ]:
from neuralmonkey.scripts.analy_syntax_good_smfr_units import plotwrap_scalar_rankwithin_regression

SAVEDIR = "/tmp"
plotwrap_scalar_rankwithin_regression(DFallpa, SAVEDIR)

### [MULT] Load regression results and plot

In [ ]:
from neuralmonkey.analyses.regression_good import all_pairwise_balanced_marginal_tests

In [ ]:
from neuralmonkey.scripts.analy_syntax_good_smfr_units import plotwrap_scalar_rankwithin_regression_MULT

for animal in ["Diego", "Pancho"]:
    
    if animal=="Diego":
        datelist=(230726, 230815, 230816, 230817, 230913, 230915, 231118) 
    elif animal == "Pancho":
        datelist=(220909, 230824, 230829, 231114, 231116, 250322) 
    else:
        assert False

    for date in datelist:

        try:
            plotwrap_scalar_rankwithin_regression_MULT(animal, date)
        except Exception as err:    
            continue